<a href="https://colab.research.google.com/github/yoonwanggyu/Self_Study/blob/main/%EC%98%A4%ED%94%84%EB%9D%BC%EC%9D%B8/GoogLeNet_%EB%85%BC%EB%AC%B8_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GoogleNet 구조

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt

Inception module

In [23]:
from tensorflow.keras import layers

def inception(input, n_1x1, n_3x3r, n_3x3, n_5x5r, n_5x5, n_pool, idx_layer):
    name_layer = f"inception_{idx_layer}"

    # 1x1 컨볼루션
    conv_1x1 = layers.Conv2D(filters=n_1x1, kernel_size=(1, 1), padding='same', activation='relu', name=f'{name_layer}/1x1')(input)

    # 3x3 컨볼루션
    conv_3x3_reduce = layers.Conv2D(filters=n_3x3r, kernel_size=(1, 1), padding='same', activation='relu', name=f'{name_layer}/3x3_reduce')(input)
    conv_3x3 = layers.Conv2D(filters=n_3x3, kernel_size=(3, 3), padding='same', activation='relu', name=f'{name_layer}/3x3')(conv_3x3_reduce)

    # 5x5 컨볼루션
    conv_5x5_reduce = layers.Conv2D(filters=n_5x5r, kernel_size=(1, 1), padding='same', activation='relu', name=f'{name_layer}/5x5_reduce')(input)
    conv_5x5 = layers.Conv2D(filters=n_5x5, kernel_size=(5, 5), padding='same', activation='relu', name=f'{name_layer}/5x5')(conv_5x5_reduce)

    # 풀링
    pool = layers.MaxPooling2D(pool_size=(3, 3), strides=1, padding='same', name=f'{name_layer}/pool')(input)
    pool_proj = layers.Conv2D(filters=n_pool, kernel_size=(1, 1), padding='same', activation='relu', name=f'{name_layer}/pool_proj')(pool)

    # 필터 결합
    output = layers.Concatenate(axis=-1, name=f'{name_layer}/output')([conv_1x1, conv_3x3, conv_5x5, pool_proj])

    return output


In [25]:
def auxilary_cls(input,name=None):
    x = layers.AveragePooling2D(pool_size=(5,5),strides=(3,3))(input)
    x = layers.Conv2D(128,(1,1),strides=1,padding='same',activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256,activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(10,activation='softmax')(x)
    return x

In [39]:
def create_googlenet():
    # creates GoogLeNet a.k.a. Inception v1 (Szegedy, 2015)
    input = tf.keras.Input(shape=(224, 224, 3))

    # conv 1 + maxpool 1
    x = layers.Conv2D(64,(7,7),strides=(2,2),activation='relu',padding='same')(input)
    x = layers.MaxPooling2D((3,3),strides=2,padding='same')(x)
    x = layers.Lambda(tf.nn.local_response_normalization)(x)

    # conv 2 (1x1 : 64, 3x3 : 192) + maxpool 2
    x = layers.Conv2D(64,(1,1),strides=1,activation='relu',padding='same')(x)
    x = layers.Conv2D(192,(3,3),strides=1,activation='relu',padding='same')(x)
    x = layers.Lambda(tf.nn.local_response_normalization)(x)
    x = layers.MaxPooling2D((3,3),strides=2,padding='same')(x)

    # Inception 3a ############################################################
    x = inception(x,64,96,128,16,32,32,'3a')
    x = inception(x,128,128,192,32,96,64,'3b')
    x = layers.MaxPooling2D((3,3),strides=2,padding='same')(x)

    # Inception 4a/4b ############################################################
    x = inception(x,192,96,208,16,48,48,'4a')
    x = inception(x,160,112,224,24,64,64,'4b')

        # auxiliary classifier **********************
    aux_1 = auxilary_cls(x)

    # Inception 4c/4d/4e ############################################################
    x = inception(x,128,128,256,24,64,64,'4c')
    x = inception(x,112,144,288,32,64,64,'4d')
    x = inception(x,256,160,320,32,128,128,'4e')
    x = layers.MaxPooling2D((3,3),strides=2,padding='same')(x)

        # auxiliary classifier **********************
    aux_2 = auxilary_cls(x)

    # Inception 5a/5b ############################################################
    x = inception(x,256,160,320,32,128,128,'5a')
    x = inception(x,384,192,384,48,128,128,'5b')
    x = layers.AveragePooling2D((7,7),strides=1,padding='same')(x)

    x = layers.Flatten()(x)
    x = layers.Dropout(0.4)(x)

    output = layers.Dense(10,activation='softmax')(x)

   # Final Model : input + output[aux.cls1, aux.cls2, main.cls]
    model = tf.keras.Model(inputs=input, outputs=[aux_1,aux_2,output])

    return model

In [40]:
model = create_googlenet()

In [41]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_39 (Conv2D)          (None, 112, 112, 64)         9472      ['input_13[0][0]']            
                                                                                                  
 max_pooling2d_30 (MaxPooli  (None, 56, 56, 64)           0         ['conv2d_39[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 lambda_14 (Lambda)          (None, 56, 56, 64)           0         ['max_pooling2d_30[0][0]